In [1]:
import pandas as pd

# Load the datasets
age_by_category = pd.read_csv('cleaned_age_by_category.csv')
death_by_smoking = pd.read_csv('cleaned_death_by_smoking.csv')
types_of_diseases = pd.read_csv('cleaned_types_of_diseases.csv')
gender_data = pd.read_csv('cleaned_gender.csv')

# Print column names to identify overlapping columns
print("Age by Category Columns:", age_by_category.columns)
print("Death by Smoking Columns:", death_by_smoking.columns)
print("Types of Diseases Columns:", types_of_diseases.columns)
print("Gender Data Columns:", gender_data.columns)

# Assuming 'Country' is the common column for merging
# Check for any overlapping column names that are not 'Country' and rename them
# Example: renaming columns in death_by_smoking if needed
death_by_smoking.rename(columns={'2021': 'Smoking_2021'}, inplace=True)

# Merging the datasets
from functools import reduce

# List of dataframes to merge
data_frames = [age_by_category, death_by_smoking, types_of_diseases, gender_data]

# Merge all dataframes on 'Country' column using an inner join
df_merged = reduce(lambda  left,right: pd.merge(left, right, on=['Country'],
                                            how='inner'), data_frames)

# Inspect the merged DataFrame
print(df_merged.head())


Age by Category Columns: Index(['Country/area', '1990', '2021.0', 'Absolute Change', 'Relative Change',
       '1990.1', '2021.0.1', 'Absolute Change.1', 'Relative Change.1',
       '1990.2', '2021.0.2', 'Absolute Change.2', 'Relative Change.2'],
      dtype='object')
Death by Smoking Columns: Index(['Albania', '137.5', '94.7', '-42.8', '-0.31'], dtype='object')
Types of Diseases Columns: Index(['Afghanistan', '25371', '1052', '7478', '296', '7168', '3125', '3144',
       '4372', '717', '2764', '25968', '8312', '6286', '13837', '30644',
       '12218', '2449', '588', '810', '687', '3931', '1077', '609', '4502',
       '1873', '2517', '11108', '14025'],
      dtype='object')
Gender Data Columns: Index(['Afghanistan', '0.394', '0.072', '39068978'], dtype='object')


KeyError: 'Country'

In [4]:
import pandas as pd
from functools import reduce

# Load the datasets assuming headers need to be defined for some
age_by_category = pd.read_csv('cleaned_age_by_category.csv')
death_by_smoking = pd.read_csv('cleaned_death_by_smoking.csv', header=None)
types_of_diseases = pd.read_csv('cleaned_types_of_diseases.csv', header=None)
gender_data = pd.read_csv('cleaned_gender.csv', header=None)

# Assuming correct headers (verify and adjust according to your data)
death_by_smoking.columns = ['Country', 'Smoking_1990', 'Smoking_2021', 'Change', 'Percent_Change']
types_of_diseases.columns = ['Country'] + [f'Disease{i}' for i in range(1, 29)]  # 29 columns in total
gender_data.columns = ['Country', 'Metric1', 'Metric2', 'Population']

# Rename 'Country/area' to 'Country' in age_by_category
age_by_category.rename(columns={'Country/area': 'Country'}, inplace=True)

# List of dataframes to merge
data_frames = [age_by_category, death_by_smoking, types_of_diseases, gender_data]

# Merge all dataframes on 'Country' column using an inner join
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Country'], how='inner'), data_frames)

# Inspect the merged DataFrame
print(df_merged.head())


     Country  1990  2021.0  Absolute Change  Relative Change  1990.1  \
0  Singapore   9.9     3.6             -6.3            -0.64   692.3   
1    Ireland  14.5     4.7             -9.8            -0.68  1702.7   
2      Malta  12.9     7.9             -5.0            -0.39   755.5   
3     Brunei  16.9    11.6             -5.3            -0.31  1373.8   
4    Burundi  21.0     6.4            -14.6            -0.70   542.3   

   2021.0.1  Absolute Change.1  Relative Change.1  1990.2  ...  Disease22  \
0     187.4             -505.0              -0.73   224.4  ...        1.0   
1     479.4            -1223.3              -0.72   416.1  ...        1.0   
2     248.8             -506.6              -0.67   285.8  ...        0.0   
3     484.8             -889.0              -0.65   347.7  ...        0.0   
4     191.4             -350.9              -0.65   273.3  ...      205.0   

   Disease23  Disease24  Disease25  Disease26  Disease27  Disease28  Metric1  \
0        0.0     1547.0 

In [5]:
# Save the merged DataFrame to a CSV file
df_merged.to_csv('merged_data.csv', index=False)
